In [1]:
from meshplot import plot
import numpy as np
import numpy.linalg as la
import igl
import time, math

In [2]:
from numba import cuda, jit, prange, njit, int32, float32, void

In [3]:
v, f = igl.read_triangle_mesh('data/cylinder.obj')

In [4]:
plot(v, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [5]:
def voxel_points(axes):
    """generate n-dimensional grid coordinates from the given set of (x, y, z...) values"""
    ndims = len(axes)
    dims = [len(x) for x in axes]
    indices = np.indices(dims)
    points = np.array([axis[index] for axis, index in zip(axes, indices)])
    points = points.reshape([ndims, np.prod(dims)])
    return points.T.copy()

In [6]:
def sample_mesh_interior(V, F, gridLength, dtype=np.float32):
    minPt = np.min(V, 0)
    maxPt = np.max(V, 0)
    q = voxel_points((np.arange(minPt[0], maxPt[0] + gridLength, gridLength, dtype=dtype),
                       np.arange(minPt[1], maxPt[1] + gridLength, gridLength, dtype=dtype),
                         np.arange(minPt[2], maxPt[2] + gridLength, gridLength, dtype=dtype)))
    #print(q)
    w = igl.fast_winding_number_for_meshes(V.astype(dtype), F, q)
    #print(w.shape)
    return q[w > 0.5]

@njit (parallel=True)
def compute_inertia_tensor(points, pointmass):
    inertia_tensor = np.zeros((3, 3), dtype=points.dtype)
    N = points.shape[0]
    for i in prange(N):
        pointCM = points[i,:]
        moments = pointCM * pointCM
        inertia_tensor[0, 0] += moments[1] + moments[2]
        inertia_tensor[1, 1] += moments[0] + moments[2]
        inertia_tensor[2, 2] += moments[0] + moments[1]
        inertia_tensor[0, 1] = pointCM[0] * pointCM[1]
        inertia_tensor[0, 2] = pointCM[0] * pointCM[2]
        inertia_tensor[1, 2] = pointCM[1] * pointCM[2]
    inertia_tensor *= pointmass
    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]
    return inertia_tensor

In [7]:
class PhysObject:
    def __init__(self, V, F, gridDim, density=1):
        self.points = sample_mesh_interior(V, F, gridDim)
        pointmass = density * gridDim ** 3
        cm = np.mean(self.points, 0)
        self.points -= cm
        self.mass = pointmass * self.points.shape[0]
        self.inertia_tensor = compute_inertia_tensor(self.points, pointmass)
        self.inertia_tensor_inv = la.inv(self.inertia_tensor)

gridDim = 0.02
start = time.time()
physobj = PhysObject(v, f, gridDim)
end = time.time()
print(end-start,'seconds with',physobj.points.shape[0],'points')

In [8]:
@njit
def hamilton_product(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@njit
def quaternion_to_matrix(q):
    s = q[0]
    vx = q[1]
    vy = q[2]
    vz = q[3]
    vx2 = vx*vx
    vy2 = vy*vy
    vz2 = vz*vz
    return np.array([[1-2*vy2-2*vz2, 2*vx*vy-2*s*vz, 2*vx*vz+2*s*vy],
                     [2*vx*vy+2*s*vz, 1-2*vx2-2*vz2, 2*vy*vz-2*s*vx],
                     [2*vx*vz-2*s*vy, 2*vy*vz+2*s*vx, 1-2*vx2-2*vy2]], dtype=q.dtype)

@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def hamilton_product_device(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def cross_product_device(p, q, out):
    a0 = p[1] * q[2] - p[2] * q[1]
    a1 = p[2] * q[0] - p[0] * q[2]
    a2 = p[0] * q[1] - p[1] * q[0]
    out[0] = a0
    out[1] = a1
    out[2] = a2

@njit
def cross_product(p, q, out):
    out0 = p[1] * q[2] - p[2] * q[1]
    out1 = p[2] * q[0] - p[0] * q[2]
    out2 = p[0] * q[1] - p[1] * q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    
@cuda.jit(device=True)
def transform(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product_device(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product_device(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

In [9]:
from numpy.random import randn
@cuda.jit
def test_cross_product_kernel(p, q, out):
    i = cuda.grid(1)
    if i == 0:
        ploc = cuda.local.array(3, float32)
        qloc = cuda.local.array(3, float32)
        outloc = cuda.local.array(3, float32)
        for j in range(3):
            ploc[j] = p[j]
            qloc[j] = q[j]
        cross_product_device(ploc, qloc, outloc)
        out[0] = outloc[0]
        out[1] = outloc[1]
        out[2] = outloc[2]
def test_cross_product(p, q):
    out = cuda.device_array(3, np.float32)
    test_cross_product_kernel[1, 1](cuda.to_device(p.astype(np.float32)), cuda.to_device(q.astype(np.float32)), out)
    return out.copy_to_host() - np.cross(p, q)

for i in range(100):
    err = la.norm(test_cross_product(randn(3), randn(3)))
    if err > 1e-6:
        print('failed with error',err)

In [43]:
cell_size = 2
    
@cuda.jit
def populate_grid_kernel(points1, pose1, points2, pose2, grid, minPt, gridDim):
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(7, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(7):
            pose[j] = pose1[j]
        offset = 0
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(7):
            pose[j] = pose2[j]
        offset = cell_size
    else:
        return
    
    transform(transpoint, pose, transpoint)
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    if px >= 0 and px < grid.shape[0] and py >= 0 and py < grid.shape[1] and pz >= 0 and pz < grid.shape[2]:
        for j in range(cell_size):
            if grid[px, py, pz * 2 * cell_size + offset + j] < 0:
                grid[px, py, pz * 2 * cell_size + offset + j] = i
                break
                
                
@cuda.jit
def compute_forces(outForces1, points1, pose1, outForces2, points2, pose2, grid, minPt, gridDim, springK, damping):
    radius = gridDim # maximum distance between interacting particles
    radius2 = radius * radius
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(13, float32)
    otherpose = cuda.local.array(13, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(13):
            pose[j] = pose1[j]
            otherpose[j] = pose2[j]
        offset = 0
        otherOffset = cell_size
        otherpoints = points2
        outForces = outForces1
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(13):
            pose[j] = pose2[j]
            otherpose[j] = pose1[j]
        offset = cell_size
        otherOffset = 0
        otherpoints = points1
        outForces = outForces2
    else:
        return
    
    transform(transpoint, pose, transpoint)
    point_cm = cuda.local.array(3, float32)
    for k in range(3):
        point_cm[k] = transpoint[k+1] - pose[k+4]
    #print('transpoint:',transpoint[1], transpoint[2], transpoint[3])
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    if px >= 0 and px < grid.shape[0] and py >= 0 and py < grid.shape[1] and pz >= 0 and pz < grid.shape[2]:
        #print('cell:',px, py, pz)
        for j in range(cell_size):
            found = False
            #if grid[px, py, pz * 2 * cell_size + offset + j] >= 0:
                #print('indices:',i, grid[px, py, pz * 2 * cell_size + offset + j])
            if grid[px, py, pz * 2 * cell_size + offset + j] == i:
                found = True
                break
        if not found:
            return
        force = cuda.local.array(3, float32)
        tempforce = cuda.local.array(3, float32)
        torque = cuda.local.array(3, float32)
        temptorque = cuda.local.array(3, float32)
        omega = cuda.local.array(3, float32)
        otheromega = cuda.local.array(3, float32)
        for l in range(3):
            force[l] = 0
            torque[l] = 0
            omega[l] = pose[l+7]
            otheromega[l] = otherpose[l+7]
        
        #print('otheromega:',otheromega[0], otheromega[1], otheromega[2])#, 'othervel:',othervel[0], othervel[1], othervel[2])
            
        vel = cuda.local.array(3, float32)
        #print('velbefore:',vel[0], vel[1], vel[2])
        cross_product_device(omega, point_cm, vel)
        #print('omega:',omega[0], omega[1], omega[2],
        #      'point_cm:',point_cm[0], point_cm[1], point_cm[2],'vel:',vel[0], vel[1], vel[2])
        
        otherpoint = cuda.local.array(4, float32)
        otherpoint[0] = 0
        otherpoint_cm = cuda.local.array(3, float32)
        othervel = cuda.local.array(3, float32)
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                for dz in range(-1, 2):
                    for j in range(cell_size):
                        otherIndex = grid[px+dx,py+dy,(pz+dz)*2*cell_size+otherOffset+j]
                        if otherIndex >= 0:
                            for k in range(3):
                                otherpoint[k+1] = otherpoints[otherIndex, k]
                            transform(otherpoint, otherpose, otherpoint)
                            for k in range(3):
                                otherpoint_cm[k] = otherpoint[k+1] - otherpose[k+4]
                                #otherpoint holds the displacement from otherpoint to this point
                                otherpoint[k+1] = transpoint[k+1] - otherpoint[k+1]                                
                            
                            dist2 = otherpoint[1]*otherpoint[1]+otherpoint[2]*otherpoint[2]+otherpoint[3]*otherpoint[3]
                            dist = math.sqrt(dist2)
                            #print(dist)
                            #spring force
                            if dist2 > 0 and dist2 < radius2:
                                    springfac = springK * (radius - dist)/dist
                                    for k in range(3):
                                        tempforce[k] = springfac * otherpoint[k+1]
                                        #print('realvalue:',springfac * otherpoint[k+1],'got:',tempforce[k])
                                        #force[k] += tempforce[k]
                                    cross_product_device(point_cm, tempforce, temptorque)
                                    for k in range(3):
                                        force[k] += tempforce[k]
                                        torque[k] += temptorque[k]
                                        
                            #damping force
                            #compute relative velocity of points
                            cross_product_device(otheromega, otherpoint_cm, othervel)
                            for k in range(3):
                                tempforce[k] = damping * (vel[k] - othervel[k])
                            cross_product_device(point_cm, tempforce, temptorque)
                            for k in range(3):
                                force[k] += tempforce[k]
                                torque[k] += temptorque[k]
                            
        outForces[i, 0] = force[0]
        outForces[i, 1] = force[1]
        outForces[i, 2] = force[2]
        outForces[i, 3] = torque[0]
        outForces[i, 4] = torque[1]
        outForces[i, 5] = torque[2]

TPB = 32


@cuda.jit
def reduce_forces_torques(d_accum, d_f):
    i = cuda.grid(1)
    tIdx = cuda.threadIdx.x
    n = d_f.shape[0]
    sh_w = cuda.shared.array((TPB, 6), dtype=float32)
    #for j in range(6):
    #    sh_w[tIdx, j] = 0
    if i < n:
        for j in range(6):
            sh_w[tIdx, j] = d_f[i, j]
    if i < 6:
        d_accum[i] = 0
    cuda.syncthreads()
    if tIdx < 6:
        component_sum = 0.0
        for j in range(cuda.blockDim.x):
            component_sum += sh_w[j, tIdx]
        cuda.atomic.add(d_accum, tIdx, component_sum)

def simulate_rigid_bodies(v1, f1, p1, m1, v2, f2, p2, m2, gridLen, minPt, maxPt, springK, damping, dt, steps):
    
    start = time.time()
    objects = [PhysObject(v, f, gridLen, m) for v, f, m in zip((v1, v2), (f1, f2), (m1, m2))]
    end = time.time()
    print('intiailized objects in',end-start)
    
    print('points in obj1:',objects[0].points.shape[0])
    print('points in obj2:',objects[1].points.shape[0])
    
    #minPt = np.minimum(np.min(v1, 0), np.min(v2, 0))
    #maxPt = np.maximum(np.max(v1, 0), np.max(v2, 0))
    
    res = np.ceil(((maxPt - minPt) / gridLen)).astype(np.int32)
    
    res[2] *= cell_size * 2
    
    start = time.time()
    
    grid = cuda.to_device(np.full(res, -1, dtype=np.int32))
    #quaternion as [wxyz], followed by translation [xyz]
    states = []
    states_d = []
    points_d = []
    forces_d = [cuda.device_array((obj.points.shape[0], 6), np.float32) for obj in objects] #[force, torque]
    forces_accum_d = [cuda.device_array(6, np.float32) for i in range(2)]
    
    for i in range(2):
        state = np.zeros((13,), dtype=np.float32)
        state[:7] = (p1, p2)[i]
        states.append(state)
        #print(state)
        states_d.append(cuda.to_device(state))
        points_d.append(cuda.to_device(objects[i].points))
        
    minPt_d = cuda.to_device(minPt)
    end = time.time()
    print('copied data to GPU in',end-start)
    
    
    m = np.sum([obj.points.shape[0] for obj in objects])
    gridDim = (m + TPB - 1) // TPB
    
    #net conserved state
    L = [np.zeros(3) for i in range(2)]
    p = [np.zeros(3) for i in range(2)]
    
    allstates = []
    
    starte = cuda.event()
    ende = cuda.event()
    
    for i in range(steps):
        starte.record()
        populate_grid_kernel[gridDim, TPB](points_d[0], states_d[0], points_d[1], states_d[1], grid, minPt_d, gridLen)
        ende.record()
        ende.synchronize()
        print('populated grid in',cuda.event_elapsed_time(starte, ende)/1000)
        #debug
        #return grid.copy_to_host()
        
        starte.record()
        compute_forces[gridDim, TPB](forces_d[0], points_d[0], states_d[0], forces_d[1], points_d[1], states_d[1], grid, minPt_d, gridLen, springK, damping)
        ende.record()
        ende.synchronize()
        print('computed individual point forces in',cuda.event_elapsed_time(starte, ende)/1000)
        #debug
        #return forces_d[0].copy_to_host(), forces_d[1].copy_to_host()
        
        for j in range(2):
            gridDimi = (objects[j].points.shape[0] + TPB - 1) // TPB
            starte.record()
            reduce_forces_torques[gridDimi, TPB](forces_accum_d[j], forces_d[j])
            ende.record()
            ende.synchronize()
            print('reduced forces',j,'on gpu in',cuda.event_elapsed_time(starte, ende)/1000)
            forces_accum = forces_accum_d[j].copy_to_host()
            
            #debug
            allforces = forces_d[j].copy_to_host()
            start = time.time()
            cpu_force = np.sum(allforces, 0)
            end = time.time()
            print('reduced forces',j,'on cpu in',end-start)
            print('force_gpu:',forces_accum,'force_cpu:',cpu_force)
            
            p[j] += forces_accum[:3] * dt
            L[j] += forces_accum[3:] * dt
            
            #compute auxiliary state
            v = p[j] / objects[j].mass
            
            R = quaternion_to_matrix(states[j][:4])
            Iinv = R @ objects[j].inertia_tensor_inv @ R.T
            omega = Iinv @ L[j]
            
            qdot = np.array([0, omega[0], omega[1], omega[2]], np.float32)
            hamilton_product(qdot, states[j][:4], qdot)
            qdot *= 0.5
            
            #timestep update
            states[j][:4] += qdot * dt
            states[j][4:7] += v * dt
            states[j][7:10] = omega
            states[j][10:13] = v
            states_d[j] = cuda.to_device(states[j])
            
        allstates.append([state.copy() for state in states])
    return allstates
            

In [44]:
pose1 = np.array([1, 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, 0, math.sqrt(2)/2, 0, 0, 2.1, 0], np.float32)
forces = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 1, 1, 0.01, 1)

intiailized objects in 1.0152842998504639
points in obj1: 781953
points in obj2: 781953
copied data to GPU in 0.02991938591003418
populated grid in 0.0010537279844284057
computed individual point forces in 0.003735071897506714
reduced forces 0 on gpu in 0.0009817919731140137
reduced forces 0 on cpu in 0.00897669792175293
force_gpu: [0. 0. 0. 0. 0. 0.] force_cpu: [0. 0. 0. 0. 0. 0.]
reduced forces 1 on gpu in 0.0005862399935722351
reduced forces 1 on cpu in 0.008975744247436523
force_gpu: [0. 0. 0. 0. 0. 0.] force_cpu: [0. 0. 0. 0. 0. 0.]


plot(np.indices(grid.shape)[:,grid >= 0].T)

In [24]:
forces[0][:,:3].sum()

-4.127196

In [25]:
forces[1][:,:3].sum()

4.127197